In [107]:
import torch
from datasets.car_evaluation import CarEvaluation
from binhd.embeddings import CategoricalEncoder
from binhd.datasets import BaseDataset
from torch.utils.data import DataLoader
from module.record_encoder import RecordEncoder
from binhd.classifiers import BinHD
import torchmetrics

In [108]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [109]:
dataset = CarEvaluation()
dimension = 1000
num_levels = 100
batch_size = 1000

In [110]:
from sklearn.calibration import LabelEncoder


X = dataset.features[dataset.categorical_features]
num_categories_per_feature = [X[col].nunique() for col in X.columns]
#print(num_categories_per_feature)

y = dataset.targets
le = LabelEncoder()
y_encoded = torch.tensor(le.fit_transform(y))

print(y_encoded)

categorical_encoder = CategoricalEncoder(dimension)
X = categorical_encoder.fit_transform(X)
#y = categorical_encoder.fit_transform(y)
#print(X)

tensor([2, 2, 2,  ..., 2, 1, 3])


/home/marinapiragibe/.local/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [116]:
from sklearn.model_selection import train_test_split


y_encoded = torch.tensor(y_encoded).to(device)

print("Distribuição das classes:")
print(torch.bincount(y_encoded))

Distribuição das classes:
tensor([ 384,   69, 1210,   65])


/tmp/ipykernel_100520/1418773981.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_encoded = torch.tensor(y_encoded).to(device)


In [112]:
with torch.no_grad():
    samples = torch.tensor(X.values).to(device)
    labels = torch.tensor(y_encoded).to(device)
    X_hv = categorical_encoder(samples)


/tmp/ipykernel_100520/1562505869.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_encoded).to(device)


In [113]:
model = BinHD(dimension, dataset.num_classes)

In [114]:
import torchhd


X_hv_combined = torchhd.multibundle(X_hv) 

X_train, X_test, y_train, y_test = train_test_split(X_hv_combined, labels, test_size=0.3, random_state = 0) 
print(X_train.shape)
print(y_train.shape)
print(X_train)


torch.Size([1209, 1000])
torch.Size([1209])
BSCTensor([[1, 1, 0,  ..., 1, 0, 1],
           [0, 1, 0,  ..., 0, 1, 0],
           [0, 1, 1,  ..., 0, 0, 1],
           ...,
           [0, 1, 1,  ..., 1, 1, 1],
           [0, 1, 1,  ..., 1, 0, 1],
           [0, 1, 0,  ..., 1, 0, 0]], dtype=torch.int8)


In [115]:
from sklearn.metrics import accuracy_score


with torch.no_grad():
    model.fit(X_train,y_train)
    predictions = model.predict(X_test.to(torch.int8))  
    acc = accuracy_score(predictions, y_test)
    print("BinHD: Accuracy = ", acc)

    model.fit_adapt(X_train,y_train)
    predictions = model.predict(X_test)  
    acc = accuracy_score(predictions, y_test)
    print("BinHD - Adapt: Accuracy = ", acc)

BinHD: Accuracy =  0.15414258188824662


fit:   0%|          | 0/30 [00:00<?, ?it/s]

fit: 100%|██████████| 30/30 [00:00<00:00, 52.63it/s]

BinHD - Adapt: Accuracy =  0.18497109826589594
